In [55]:
import pandas as pd
import numpy as np
from konlpy.tag import Okt

In [3]:
df = pd.read_excel("collect data/강남구_finished.xlsx")

In [41]:
# 불용어 제거 (자음이나 모음만 들어가있는 이상한 말들 제거, 숫자 제거)
df["리뷰"] = df["리뷰"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

/var/folders/5v/6byb3xz548v1xd0b83z24ys40000gn/T/ipykernel_4216/3473102629.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["리뷰"] = df["리뷰"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [42]:
df["리뷰"].replace('', np.nan, inplace=True)
df["리뷰"] = df["리뷰"].dropna(how='any')

In [43]:
df["리뷰"]

0                                                    만두 맛집
1        진미평양 냉면 나의 평양냉면 맛집 탑 평양냉면 가게 하지만 본관이던 별관이던 너무 ...
2        평냉 최고존엄입니다처음에는 순정의 맛을 즐기다가그다음 고춧가루를 조금 풀어보시면 미...
3        어복쟁반 냉면 만두 시켜먹었는데 세가지 다 맛있어요ㅜㅜ 양도 너무 많아서 남은거 포...
4        정말정말정맣 좋아하는 곳이에요멜로가 체질 나와서 알게된 곳인데 제일 좋아하는 평냉집...
                               ...                        
32279                                   음 이건 평양냉면이라 할 수 없엉
32280                                                 굿입니다
32281                                  평양냉면 원 돼지편육중 원  년 월
32282                                                  NaN
32283    평양냉면 맛있음식사 밥류 시키면 반찬도 잘 나옴알에 천원씩 하는 만두는 엄청 큼식사...
Name: 리뷰, Length: 32284, dtype: object

In [44]:
df["리뷰"].value_counts()

맛있어요                                                                                                                                                                                                                                                                                                            289
굿                                                                                                                                                                                                                                                                                                                40
좋아요                                                                                                                                                                                                                                                                                                              39
                                                                            

In [60]:
df = df[df["리뷰"].notnull()]

In [61]:
reviews = df["리뷰"]
star = df["평점"]

In [62]:
len(reviews), len(star)

(25809, 25809)

In [63]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(reviews, star, test_size=0.2, random_state=42)

In [64]:
len(train_x), len(train_y), len(test_x), len(test_y)

(20647, 20647, 5162, 5162)

In [65]:
okt = Okt()

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(tokenizer=okt.morphs, ngram_range=(1,2), min_df=3, max_df=0.9)
tfv.fit(train_x)
tfv_train_x = tfv.transform(train_x)
print(tfv_train_x)

/Users/cmblir/miniforge3/envs/lcb/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


  (0, 31981)	0.6576439451800637
  (0, 23093)	0.2736179876460674
  (0, 17221)	0.701881498690853
  (1, 29983)	0.23555202091190283
  (1, 29971)	0.1398448044135239
  (1, 23574)	0.23959125014791444
  (1, 23297)	0.054424020401518454
  (1, 22328)	0.15820224495302554
  (1, 22282)	0.06787639636240556
  (1, 21829)	0.18923375812413396
  (1, 21828)	0.18250689638071432
  (1, 20830)	0.18250689638071432
  (1, 19198)	0.1759766643199059
  (1, 19193)	0.13690992208356917
  (1, 16477)	0.20017972500192754
  (1, 15880)	0.20017972500192754
  (1, 15017)	0.24436864434028877
  (1, 15005)	0.11914068324716787
  (1, 13955)	0.21933126034416547
  (1, 13931)	0.10936662058635852
  (1, 12309)	0.16721514737466459
  (1, 12282)	0.11723029815849209
  (1, 11246)	0.15859628430443468
  (1, 11007)	0.23959125014791444
  (1, 11005)	0.13743575490805732
  :	:
  (20646, 18762)	0.108964923314392
  (20646, 17629)	0.17371063071525092
  (20646, 17616)	0.12220452406306859
  (20646, 15670)	0.20161801025251266
  (20646, 15143)	0.165624182

In [72]:
from sklearn.linear_model import LogisticRegression # 이진 분류 알고리즘
from sklearn.model_selection import GridSearchCV # 하이퍼 파라미터 최적화

clf = LogisticRegression(random_state=42)
params = {'C': [0.01, 0.1, 1, 5, 10], 'max_iter' : [100, 500]}
grid_cv = GridSearchCV(clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfv_train_x, train_y)

# 최적의 평가 파라미터는 grid_cv.best_estimator_에 저장됨
print(grid_cv.best_params_, grid_cv.best_score_)# 가장 적합한 파라메터, 최고 정확도 확인

Fitting 3 folds for each of 10 candidates, totalling 30 fits


/Users/cmblir/miniforge3/envs/lcb/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/cmblir/miniforge3/envs/lcb/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

{'C': 5, 'max_iter': 100} 0.6082241408688522


/Users/cmblir/miniforge3/envs/lcb/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [73]:
tfv_test_x = tfv.transform(test_x)
# test_predict = grid_cv.best_estimator_.score(tfv_test_x,test_y)
test_predict = grid_cv.best_estimator_.predict(tfv_test_x)
from sklearn.metrics import accuracy_score
print('감성 분류 모델의 정확도 : ',round(accuracy_score(test_y, test_predict), 3))

감성 분류 모델의 정확도 :  0.609


In [75]:
import re

In [80]:
input_text = '시발'
#입력 텍스트에 대한 전처리 수행
input_text = re.compile(r'[ㄱ-ㅣ가-힣]+').findall(input_text)
input_text = [" ".join(input_text)]
# 입력 텍스트의 피처 벡터화
st_tfidf = tfv.transform(input_text)

# 최적 감성 분석 모델에 적용하여 감성 분석 평가
st_predict = grid_cv.best_estimator_.predict(st_tfidf)

#예측 결과 출력
if(st_predict == 0):
    print('예측 결과: ->> 부정 감성')
else :
    print('예측 결과: ->> 긍정 감성')

예측 결과: ->> 긍정 감성
